In [2]:
import csv
import math
import random

In [3]:
def loadCsv(filename):
    lines = csv.reader(open(filename))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset
    

In [4]:
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

In [5]:
def seperateByClass(dataset):
    seperated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in dataset):
            seperated[vector[-1]] = []
        seperated[vector[-1]].append(vector)
    return seperated

In [6]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [7]:
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers) -1)
    return math.sqrt(variance)


In [8]:
def summarize(dataset):
    summaries = [(mean(attributes), stdev(attributes)) for attributes in dataset]
    del summaries[-1]
    return summaries

In [9]:
def summarizeByClass(dataset):
    seperated = seperateByClass(dataset)
    summaries = {}
    for classValue, instances in seperated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [10]:
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean, 2)/(2*math.pow(stdev, 2))))
    return (1/(math.sqrt(2*math.pi) * stdev)) * exponent

In [11]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
        return probabilities

In [12]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProp = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProp:
            bestProp = probability
            bestLabel = classValue
    return bestLabel

In [13]:
def getPredictions(summary, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summary, testSet[i])
        predictions.append(result)
    return predictions

In [14]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet)))*100.0

In [15]:
def main():
    filename = 'diabetes-modified.csv'
    splitRatio = 0.2
    dataset = loadCsv(filename)
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split {0} rows into train = {1} and test = {2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
    # Prepare model
    summaries = summarizeByClass(trainingSet)
    # Test set
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: {0}%'.format(accuracy))

if __name__ == '__main__':
    main()


Split 768 rows into train = 153 and test = 615 rows
Accuracy: 34.63414634146341%
